In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier
from sklearn.model_selection import KFold,train_test_split

In [ ]:
data=pd.read_csv("Fraud_check.csv")

In [ ]:
data

In [ ]:
data["Marital.Status"].unique()# we have 3 catergory-single,divorced,married

In [ ]:
data["Undergrad"].value_counts()#there are 2 catergorical vales i.e-yes or no


In [ ]:
data["Urban"].unique()# 2 catergory-yes or no

In [ ]:
data.info()

In [ ]:
data.isna().sum()# there are no null values.

In [ ]:
data.describe()

In [ ]:

data

In [ ]:
data=data.rename({"Marital.Status":"Marital_Status"},axis=1)


In [ ]:
data=data.rename({"Taxable.Income":"Taxable_Income"},axis=1)

In [ ]:
data=data.rename({"City.Population":"City_Population"},axis=1)

In [ ]:
data

In [ ]:
data["Taxable_inx"]=pd.cut(data["Taxable_Income"],bins = [10000,30000,99619],labels=["risky","good"])

In [ ]:
data

In [ ]:
data=pd.get_dummies(data,columns=['Undergrad','Marital_Status','Urban'], drop_first=True)

In [ ]:
data

In [ ]:
data.Taxable_inx.unique()

In [ ]:
data

In [ ]:
data1=data.drop(columns="Taxable_inx")

In [ ]:
data1

In [ ]:
import seaborn as sns
sns.pairplot(data=data1,hue="Taxable_Income")

In [ ]:
def function(i):
  i=(i-i.min())/(i.max()-i.min())
  return i

In [ ]:
data1_norm=function(data1)
data1_norm

In [ ]:
import matplotlib.pyplot as plt
sns.barplot(data1_norm["Taxable_Income"],data1_norm["City_Population"])

In [ ]:
#as we can see or assume that either the city population is increasing or decrease the income is not effecting.

In [ ]:
sns.scatterplot(data1_norm["Taxable_Income"],data1_norm["Urban_YES"])

In [ ]:
#urban location is also not impacted for income catergory.

In [ ]:
#we will delete the population,Taxable income  and urban features from dataset.
data1_norm=data1_norm.drop(columns="Taxable_Income")


In [ ]:
data1_norm=data1_norm.drop(columns="City_Population")
data1_norm=data1_norm.drop(columns="Urban_YES")

In [ ]:
#dividing the dataset into X,Y -dependent and indepenent variable
X=data1_norm
Y=data["Taxable_inx"]

In [ ]:
Y.unique()

In [ ]:
#for testing the accuaracy we will split into test train dataset.
xtrain,xtest,ytrain,ytest=train_test_split(X,Y,test_size=0.3,random_state=1)

In [ ]:
#building the model
model=RandomForestClassifier(n_estimators=100,criterion="gini",max_features=3)
model.fit(xtrain,ytrain)
ypred=model.predict(xtest)

In [ ]:
(ypred==ytest).value_counts()#128/(128+52)=0.71~71% accuaracy

In [ ]:
from sklearn.metrics import confusion_matrix
cf=confusion_matrix(ytest,ypred)

In [ ]:
cf

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(ytest,ypred))

In [ ]:
# we find the average accuracy by kfold method.
from sklearn.model_selection import KFold,cross_val_score
kfold=KFold(n_splits=10)
results=cross_val_score(model,X,Y,cv=kfold)


In [ ]:
results.mean()# average mean of all 10 model is 0.70~70%


In [ ]:
# we will try to increase the accuracy by bagging method
from sklearn.ensemble import BaggingClassifier
model1=BaggingClassifier(base_estimator=RandomForestClassifier(n_estimators=110,criterion="gini",max_features=3))
results1=cross_val_score(model1,X,Y,cv=kfold)

In [ ]:
results1.mean()